In [1]:
import gradio as gr
from typing import Optional, List, Any
import torch
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms.base import LLM
from langchain.callbacks.manager import CallbackManagerForLLMRun
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig, TextIteratorStreamer, AutoProcessor
# from modelscope import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_core.outputs.generation import GenerationChunk
from torch import device
import time
from langchain.memory import ConversationBufferMemory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.chains.history_aware_retriever import create_history_aware_retriever

from langchain.chains.retrieval import create_retrieval_chain
from langchain_core.runnables import RunnableWithMessageHistory
from langchain.chains.combine_documents import create_stuff_documents_chain


from threading import Thread
import warnings
warnings.filterwarnings("ignore")

/root/miniconda3/lib/python3.8/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/root/miniconda3/lib/python3.8/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issu

In [2]:
from modelscope import AutoModelForCausalLM, AutoTokenizer


2024-11-02 11:38:48,213 - modelscope - INFO - PyTorch version 2.0.0+cu118 Found.
2024-11-02 11:38:48,214 - modelscope - INFO - Loading ast index from /root/.cache/modelscope/ast_indexer
2024-11-02 11:38:48,290 - modelscope - INFO - Loading done! Current index file version is 1.9.5, with md5 519743e9af8b874f864c8480530f7cdb and a total number of 945 components indexed


In [3]:
# 构建模型
class QianWenLLM(LLM):
    # 基于本地的QianWen7B-Chat模型自定义LLM类
    tokenizer: AutoTokenizer = None
    model: AutoModelForCausalLM = None
    processor: AutoProcessor = None
    
    def __init__(self, model_dir: str):
        # 从本地加载模型
        super().__init__()
        print('正从本地加载模型。。。。。')

        self.tokenizer = AutoTokenizer.from_pretrained(
            pretrained_model_name_or_path=model_dir,
            trust_remote_code=True,
        )
        self.model = AutoModelForCausalLM.from_pretrained(
            pretrained_model_name_or_path=model_dir,
            device_map='auto',
            trust_remote_code=True,
            # torch_dtype=torch.bfloat16,
            # temperature=0.1
            )
        self.model = self.model.eval()
        self.model.generation_config = GenerationConfig.from_pretrained(
            model_dir,
            trust_remote_code=True
        )
        # 可指定不同的生成长度、top_p等相关超参
        self.processor = AutoProcessor.from_pretrained(model_dir)
        print('模型加载完成！')

    def _call(self, messages,
              stop: Optional[List[str]] = None,
              run_manager: Optional[CallbackManagerForLLMRun] = None,
              **kwargs: Any):
        # response, history = self.model.chat(self.tokenizer, prompt, history=[])
        # print('打印messages内容：', messages)
        system_info =  messages.split('System:')[-1].split('Human:')[0].strip()
        human_info =  messages.split('Human:')[-1].strip()
        contextualize_question_prompt = [
            {'role':'system', 'content': system_info},
            {'role':'user', 'content': human_info}
            ]
        # print('打印contextualize_question_prompt内容：', contextualize_question_prompt)
        text = self.tokenizer.apply_chat_template(
            contextualize_question_prompt,
            tokenize=False,
            add_generation_prompt=True
        )
        model_inputs = self.tokenizer([text], return_tensors='pt').to('cpu')
        generated_ids = self.model.generate(
            **model_inputs,
            max_new_tokens=512
        )
        generated_ids = [
            output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]

        response = self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
        print('模型响应结果：', response)
        
        return response
        
    @property
    def _llm_type(self) -> str:
        return "QwenLM"

In [4]:
# 构建改写问题proompt
def contextualize_question_prompt():
    system_prompt = """\
    请根据聊天历史和最后用户的问题，改写用户最终剔除的问题。
    你只需要改写用户最终的问题，请不要回答问题。
    没有聊天历史则将用户问题直接返回，有聊天历史则改写。
    请用中文回答：
    """
    contextualize_question_prompt = ChatPromptTemplate(
    [
    ('system', system_prompt),
        MessagesPlaceholder('chat_history'),
        ('human', '{input}')
    ]
    )
    # contextualize_question_prompt = [
    # {'role':'system', 'content': system_prompt},
    # MessagesPlaceholder('chat_history'),
    # {'role':'user', 'content':'{input}'}
    # ]
    return contextualize_question_prompt

# 构建正常问答prompt
def answer_prompt():
    # system_prompt = """\
    # 首先请根据用户提出的问题分类该问题是政策问题还是日常问题。政策问题分类为一号，日常问题分类为二号。
    # 其次根据分类结果利用下面的方式对问题给出回答。
    # 一号：使用上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答案，{context}。
    # 二号：仅根据用户提出的问题独立回答，不使用任何提供的其他信息。
    # 最后结果输出格式为：
    # 一号：回答;
    # 二号：回答。
    # """
    
    system_prompt = """\
    使用输入的文本信息来回答用户的问题。如果你不知道答案，就说你不知道，不要试图编造答案。
    输入的文本：{context}
    请用中文回答：
    """
    # 问题: {question}
    # 有用的回答:
    
    qa_prompt = ChatPromptTemplate(
    [
    ('system', system_prompt),
        MessagesPlaceholder('chat_history'),
        ('human', '{input}')
    ]
    )
    # qa_prompt = [
    # {'role':'system', 'content': system_prompt},
    # MessagesPlaceholder('chat_history'),
    # {'role':'user', 'content':'{input}'}
    # ]
    

    #     [
    # ('system', system_prompt),
    #     MessagesPlaceholder('chat_history'),
    #     ('human', '{input}')
    # ]
    return qa_prompt

# 构造存储函数
def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [5]:
def init_db():
    persist_directory = r'vectordb/chroma'
    embeddings_model_cache_path = r'autodl-tmp/embedding_model/Ceceliachenen/paraphrase-multilingual-MiniLM-L12-v2'
    # 加载词向量模型
    embeddings = HuggingFaceEmbeddings(
        model_name=embeddings_model_cache_path)
    # 加载缓存知识库
    vectordb = Chroma(
        persist_directory=persist_directory,
        embedding_function=embeddings,
    )
    return vectordb

In [6]:
def init_model():
    # 初始化模型
    # model_cache_path = r'autodl-tmp/Qwen/Qwen-7B-Chat'
    # model_cache_path = r'autodl-tmp/Qwen/Qwen2.5-7B-Instruct'
    model_dir = r'autodl-tmp/Qwen/Qwen2.5-7B-Instruct'
    llm = QianWenLLM(model_dir)
    return llm

In [7]:
def qa_chain(llm, vectordb, get_session_history):
    """
    构建问答链
    :param persist_directory: 知识库本地保存路径，这里初始化政策信息知识库
    :return: 返回调用LLM回答
    """
    # 因为历史记录问题进行问题改写再传到模型
    question_prompt = contextualize_question_prompt()
    # 历史对话巡回器
    history_aware_retriever = create_history_aware_retriever(llm, 
                                                            vectordb.as_retriever(), 
                                                             question_prompt)
    qa_prompt = answer_prompt()
    qa_chain = create_stuff_documents_chain(llm, qa_prompt)
    rag_chain = create_retrieval_chain(history_aware_retriever, qa_chain)
    
    # 包装
    conversational_rag_chain = RunnableWithMessageHistory(
        runnable=rag_chain,
        get_session_history=get_session_history,
        input_messages_key='input',
        history_messages_key='chat_history',
        output_messages_key='answer'
    )

    return conversational_rag_chain


In [ ]:
store = {}
model = init_model()
db = init_db()

正从本地加载模型。。。。。


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
response = qa_chain(
                model,  
                db,
                get_session_history).invoke(
                    {'input': '如果投标文件解密失败，招标人是否有义务提供补救方案？'},
                    config={'configurable':{'session_id':'test123'}}
                    )['answer']
response

In [ ]:
chat_history_ = ''
question_prompt = f"""
                请根据聊天历史和最后用户的问题，改写用户最终提出的问题。
                你只需要改写用户最终的问题，请不要回答问题。
                没有聊天历史则将用户问题直接返回，有聊天历史则改写。
                聊天历史：{chat_history_}
                """

question_messages = [
{'role':'system', 'content': question_prompt},
{'role':'user', 'content':prompt}]
print(llm(question_messages))

In [ ]:
prompt = "2023年7月，中国航天科技集团公司五院研制的长征十二号运载火箭在酒泉卫星发射中心成功发射，将一颗北斗导航卫星送入预定轨道。"
sys='''你是一名实体提取和意图识别分类领域专家，请严格遵循以下任务和工作流程的指示输出结果。

            任务：
            1-判断用户问题是否存在实体。
            2-抽取用户问题所有实体。
            3-根据实体与给出的意图标签进行判定该用户问题的意图。

            意图标签：政策知识，日常知识，招中标知识

            工作流程：
            1.-先判断是否存在实体，不存在实体则直接根据不存在实体输出格式输出，存在实体则继续以下工作流程，并通过存在实体输出格式输出。
            2-实体提取：请从用户问题中提取出所有实体。
            3-意图分类：请根据第2点提取的实体以及意图标签，进行意图识别并分类用户问题。

            不存在实体输出格式：
                实体提取:[]
                意图分类:日常知识

            存在实体输出格式：
                实体提取：[实体1, 实体2, ...]
                意图分类：意图标签

            有用的回答：'''
messages = [
        {"role": "system", "content": sys},
        {"role": "user", "content": prompt}
    ]
print(llm(messages))